In [1]:
import os
import pandas as pd

# Path to the main directory
main_dir = "../../data/Evaluation_CoTs"

# Subdirectories
subdirectories = ["claude-3-haiku-20240307", "gpt-3.5-turbo-0125","gpt-4"]

# List to store dataframes
dataframes = []

# Iterate over each subdirectory and process files
for sub in subdirectories:
    for file in os.listdir(os.path.join(main_dir, sub)):
        if file.endswith('.csv'):
            # Determine the difficulty from the file name
            difficulty = 'easy' if 'easy' in file else 'hard'
            
            # Read CSV file
            df = pd.read_csv(os.path.join(main_dir, sub, file))
            
            # Extract model from the directory name
            model = sub
            
            # Add columns for difficulty and model
            df['Difficulty'] = difficulty
            df['Model'] = model
            
            # Append the dataframe to the list
            dataframes.append(df)

# Concatenate all dataframes
final_df = pd.concat(dataframes, ignore_index=True)



In [2]:
final_df

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Unnamed: 358,Unnamed: 359,Unnamed: 360,Unnamed: 361,Unnamed: 362,Unnamed: 363,Unnamed: 364,Unnamed: 365,Unnamed: 366,Unnamed: 367
0,MathQA_dev,Math,a multiple choice test consists of 4 questions...,c,\n Step 1: The question is asking about the...,c,"[(0, 0)]",\n Step 1: The test consists of 4 questions...,c,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MathQA_dev,Math,a 3 - digit positive integer is chosen at rand...,b,Step 1: Understand the problem statement. A 3-...,a,"[(0, 0)]",Step 1: To find the probability that the produ...,b,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MathQA_dev,Math,if x and y are positive integers and 7 + x + y...,c,Step 1: The given equation is 7 + x + y + xy =...,b,"[(0, 0)]",Step 1: We are given the equation 7 + x + y + ...,b,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MathQA_dev,Math,the hcf and lcm of two numbers m and n are res...,d,Step 1: The given information is that the high...,b,"[(0, 0)]",Step 1: Given that the HCF (Highest Common Fac...,b,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MathQA_dev,Math,"in a kilometer race , a beats b by 48 meters o...",a,Step 1: We are given that A beats B by 48 mete...,d,"[(0, 0)]","Step 1: We are given that in a kilometer race,...",b,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,GSM8K_test,Math,"Out of the 200 Grade 5 students, 2/5 are boys ...",40,Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6595,GSM8K_test,Math,Harry slept 9 hours last night. His friend Jam...,3,Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6596,GSM8K_test,Math,"When Freda cooks canned tomatoes into sauce, t...",12,Step 1: Determine the volume of the canned tom...,12,"[(0, 0)]",Step 1: Determine the volume of tomatoes used ...,12,"[(0, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6597,GSM8K_test,Math,Cars have lined up on the motorway. Some of th...,5,Step 1: Denote the number of cars that drove t...,5,"[(0, 0)]",Step 1: Denote the number of cars that drove t...,5,"[(3, 0)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Drop columns whose names start with "Unnamed:"
unnamed_cols = ['index'] + [col for col in final_df.columns if col.startswith('Unnamed:')]
final_df.drop(columns=unnamed_cols, inplace=True)

In [4]:
final_df.shape

(6599, 126)

In [5]:
final_df = final_df.dropna()
final_df = final_df[final_df['Correct Answer'] != 'NO']

In [6]:
final_df.shape

(6554, 126)

In [7]:
final_df.Name.value_counts()

Name
GSM8K_hard               1098
BigBench_hard            1098
GSM8K_test               1097
MathQA_challenge_test    1090
MathQA_dev               1087
BigBench_easy            1084
Name: count, dtype: int64

In [8]:
import pandas as pd
import re

# Enhanced regex pattern to capture negative numbers, percentages, and dollar amounts
digit_re = re.compile(r"-?\$?\d+\.?\d*%?")

def clean_value(value):
    """ Helper function to clean individual values based on the enhanced regex pattern. """
    # Remove unwanted characters
    value = str(value).replace('$', '').replace('%', '')
    # Find all numbers in the string
    matches = digit_re.findall(value)
    if matches:
        # Taking the first match, remove any trailing non-numeric characters if present
        num = matches[0].rstrip('%')
        # Convert to float and back to string to normalize the number format
        cleaned_value = str(float(num))
    else:
        # If no valid numbers are found, return an invalid flag
        cleaned_value = "error"
    return cleaned_value

def clean_columns(df, columns):
    """ Cleans specified columns in the dataframe by applying the clean_value function. """
    for column in columns:
        if column in df.columns:
            df[column] = df[column].apply(clean_value)

def clean_answers(df):
    """ Cleans all specified answer columns in the dataframe including the 'Correct Answer'. """
    answer_columns = [f"Final Answer_{i}" for i in range(40)] + ["Correct Answer"]
    clean_columns(df, answer_columns)


df_gsm8k = final_df[final_df.Name.str.startswith('GSM')]

In [9]:
clean_answers(df_gsm8k)

/tmp/ipykernel_299895/3563676368.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(clean_value)


In [10]:
df_bb = final_df[final_df.Name.str.startswith('Big')]

In [11]:
def extract_options(question):
    """ Extracts the mapping of options from the question string. """
    options_text = question.split('Options:\n')[-1]
    options = re.findall(r"\((.)\)\s(.+)", options_text)
    return {opt[1].strip(): opt[0] for opt in options}

def clean_answer(answer, options):
    """ Converts direct answers to their corresponding choice labels and standardizes the case. """
    answer = answer.strip()
    if len(answer) == 1 and answer.upper() in options.values():
        return answer.upper()
    elif answer in options:
        return options[answer]
    else:
        return "erroe"  # Or any other default value for unidentified answers

def process_answers(df):
    """ Processes all answer columns based on the options extracted from the question. """
    for i in range(40): 
        answer_column = f"Final Answer_{i}"
        df[answer_column] = df.apply(lambda x: clean_answer(x[answer_column], extract_options(x['Question'])), axis=1)

In [12]:
process_answers(df_bb)

/tmp/ipykernel_299895/808227791.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[answer_column] = df.apply(lambda x: clean_answer(x[answer_column], extract_options(x['Question'])), axis=1)


In [13]:
df_math = final_df[final_df.Name.str.startswith('Math')]

In [14]:
import pandas as pd
import re

def extract_options(question):
    """ Extracts the mapping of options from the question string based on the new format. """
    options_text = question.split('The options are:')[-1]
    # Adjusting the regex pattern to match the new option format
    options = re.findall(r"(\w)\s\)\s(\d+)", options_text)
    return {opt[1].strip(): opt[0].lower() for opt in options}  # Keeping the option letters in lowercase

def clean_answer(answer, options):
    """ Converts direct answers to their corresponding choice labels and keeps the case as is (lowercase). """
    answer = answer.strip()
    # Check if answer is a number and map it back to the corresponding option letter
    if answer in options:
        return options[answer]
    # Check if the answer is already a valid option letter
    elif len(answer) == 1 and answer.lower() in options.values():
        return answer.lower()
    else:
        return "error"  # Or any other default value for unidentified answers

def process_answers(df):
    """ Processes all answer columns based on the options extracted from the question. """
    for i in range(40):  # Assuming there are 40 answer columns as mentioned
        answer_column = f"Final Answer_{i}"
        # Applying cleaning to each answer column based on the extracted options
        df[answer_column] = df.apply(lambda x: clean_answer(x[answer_column], extract_options(x['Question'])), axis=1)

process_answers(df_math)


/tmp/ipykernel_299895/379397058.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[answer_column] = df.apply(lambda x: clean_answer(x[answer_column], extract_options(x['Question'])), axis=1)


In [15]:
pd.concat([df_gsm8k, df_bb, df_math]).reset_index(drop=True).to_csv('../../data/Evaluation_CoTs/final.csv', index=False)

# Test code to run on final data

In [16]:
final = pd.concat([df_gsm8k, df_bb, df_math]).reset_index(drop=True)

In [2]:
import sys
sys.path.insert(0, '../')
from CS_feature_extractor import *
from IDV_CS_Model import trained_LR_model

/home/guangya/miniconda3/envs/env1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/guangya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39,Difficulty,Model
0,GSM8K_test,Math,Janet’s ducks lay 16 eggs per day. She eats th...,18,"Step 1: Calculate the total number of eggs, wh...",18,"[(0, 0)]","Step 1: Calculate the total number of eggs, wh...",18,"[(0, 0)]",...,18,"[(0, 0)]","Step 1: Calculate the total number of eggs, wh...",18,"[(0, 0)]","Step 1: Calculate the total number of eggs, wh...",18,"[(0, 0)]",easy,gpt-4
1,GSM8K_test,Math,A robe takes 2 bolts of blue fiber and half th...,3,Step 1: Identify the quantity of blue fiber re...,3,"[(0, 0)]",Step 1: Identify the quantity of blue fiber re...,3,"[(0, 0)]",...,3,"[(0, 0)]",Step 1: Identify the quantity of blue fiber re...,3,"[(0, 0)]",Step 1: Identify the quantity of blue fiber us...,3,"[(0, 0)]",easy,gpt-4
2,GSM8K_test,Math,Josh decides to try flipping a house. He buys...,70000,Step 1: Calculate the total cost of the house ...,"$195,000","[(0, 0)]",Step 1: Calculate the total cost of the house ...,"$195,000","[(0, 0)]",...,"$195,000","[(0, 0)]",Step 1: Calculate the total cost of the house ...,"$195,000","[(0, 0)]",Step 1: Calculate the total cost of the house ...,"$195,000","[(0, 0)]",easy,gpt-4
3,GSM8K_test,Math,James decides to run 3 sprints 3 times a week....,540,Step 1: Multiply the number of sprints (3) by ...,540,"[(0, 0)]",Step 1: Calculate the total distance James run...,540,"[(0, 0)]",...,540,"[(0, 0)]",Step 1: Multiply the number of sprints by the ...,540,"[(0, 0)]",Step 1: Calculate the total distance James run...,540,"[(0, 0)]",easy,gpt-4
4,GSM8K_test,Math,"Every day, Wendi feeds each of her chickens th...",20,Step 1: Calculate the total amount of feed by ...,20,"[(0, 0)]",Step 1: Calculate the total amount of feed nee...,20,"[(0, 0)]",...,20,"[(0, 0)]",Step 1: Calculate the total amount of feed by ...,20,"[(0, 0)]",Step 1: Calculate the total amount of feed Wen...,20,"[(0, 0)]",easy,gpt-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GSM8K_test,Math,"Out of the 200 Grade 5 students, 2/5 are boys ...",40,Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",...,40,"[(0, 0)]",Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",Step 1: Calculate the number of boys by multip...,40,"[(0, 0)]",easy,gpt-4
96,GSM8K_test,Math,Harry slept 9 hours last night. His friend Jam...,3,Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",...,3,"[(0, 0)]",Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",Step 1: Calculate James's sleep time = Harry's...,3,"[(0, 0)]",easy,gpt-4
97,GSM8K_test,Math,"When Freda cooks canned tomatoes into sauce, t...",12,Step 1: Determine the volume of the canned tom...,12,"[(0, 0)]",Step 1: Determine the volume of tomatoes used ...,12,"[(0, 0)]",...,12,"[(0, 0)]",Step 1: Determine the original volume of the t...,12,"[(0, 0)]",Step 1: Determine the volume of tomatoes used ...,12,"[(0, 0)]",easy,gpt-4
98,GSM8K_test,Math,Cars have lined up on the motorway. Some of th...,5,Step 1: Denote the number of cars that drove t...,5,"[(0, 0)]",Step 1: Denote the number of cars that drove t...,5,"[(3, 0)]",...,5,"[(0, 0)]",Step 1: Calculate the total number of cars tha...,5,"[(0, 0)]",Step 1: Calculate the total number of cars tha...,5,"[(1, 0)]",easy,gpt-4


In [4]:
sim_input = extract_sim_input(df)

In [6]:
extract_sim(df, method='bigram').shape

jaccard with bigram time cost: 4.045197010040283s


(100, 40)

In [ ]:
df_with_features = pd.DataFrame(extract_feature(final))
feature_li = [
    'LEN',
    'QUA_IM',
    'DIF_IV',
    'SIM_COT_BIGRAM',
    'SIM_COT_AGG',
    'SIM_COT_PW',
    'SIM_AC_BIGRAM',
    'SIM_AC_AGG',
    'SIM_AC_PW',
]
# coe = [-0.1,-5,-1,3,2,2,2]
# # intercept = -1.5
# df = trained_LR_model(df_with_features,feature_li)

In [ ]:
df = trained_LR_model(df_with_features,feature_li)

In [18]:
asnwers, bin = extract_cot_answer(final)

In [19]:
# Sample DataFrame and Correctness array (replace these with your actual data)
columns = [f'CoT_{i}' for i in range(40)]
data = final[columns]

# Convert Correctness array to DataFrame
correctness_df = pd.DataFrame(bin, columns=[f'CoT_{i}' for i in range(40)])

# Unpivot/melt the original DataFrame
melted_data = data.reset_index().melt(id_vars='index', var_name='CoT', value_name='CoT_Value')

# Unpivot/melt the Correctness DataFrame
melted_correctness = correctness_df.reset_index().melt(id_vars='index', var_name='CoT', value_name='Correctness_Value')

# Merge the two melted DataFrames on 'index' and 'CoT'
combined_df = pd.merge(melted_data, melted_correctness, on=['index', 'CoT'])

# Drop the 'index' and 'CoT' columns if they are no longer needed
combined_df = combined_df[['CoT_Value', 'Correctness_Value']]

# Show the result
print(combined_df.head())

                                           CoT_Value  Correctness_Value
0  Step 1: Determine the total number of eggs lai...                  1
1  Step 1: The question states that a robe takes ...                  1
2  Step 1: Given information\n- Josh bought a hou...                  0
3  Step 1: Identify the given information.\n    -...                  1
4  Step 1: Determine the total amount of feed Wen...                  1


In [20]:
combined_df.shape

(262160, 2)

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming df_concate is your DataFrame and it's correctly loaded with 'CoT_Value' as text and 'Correctness' as labels
df_concate = combined_df  # Make sure combined_df is your dataframe name that includes 'CoT_Value' and 'Correctness'

# Split index
split_idx = int(len(df_concate) * 0.8)  # 80% for training

# Split the data into training and testing
X_train = df_concate['CoT_Value'].iloc[:split_idx]
y_train = df_concate['Correctness_Value'].iloc[:split_idx]
X_test = df_concate['CoT_Value'].iloc[split_idx:]
y_test = df_concate['Correctness_Value'].iloc[split_idx:]

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512)

# Convert to PyTorch datasets
class CoTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {key: torch.tensor(val).to(device) for key, val in encodings.items()}
        self.labels = torch.tensor(labels).to(device)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CoTDataset(train_encodings, y_train.to_list())
test_dataset = CoTDataset(test_encodings, y_test.to_list())

# Load BERT with a classification head and move model to device
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

print(f"Model device: {model.device}")
print(f"Data device: {train_dataset.encodings['input_ids'].device}")





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0
Data device: cuda:0


In [25]:
import torch

# Free GPU memory
torch.cuda.empty_cache()

In [26]:
from sklearn.metrics import roc_auc_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    roc_auc = roc_auc_score(labels, preds)
    accuracy = accuracy_score(labels, preds)
    
    return {
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5000,
    dataloader_pin_memory=False,
    save_strategy='epoch',  # Save model at the end of each epoch
    save_total_limit=2,     # Keep only the last 2 saved models
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model='accuracy',  # Use accuracy as the metric for selecting the best model
    greater_is_better=True  # Higher accuracy is better
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()



/home/wxr9et/.local/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy
1,No log,0.265108,0.884207,0.884250
2,0.349300,0.207640,0.920706,0.920945
3,0.349300,0.187608,0.932907,0.932980
4,0.161200,0.196518,0.939073,0.939083


TrainOutput(global_step=13108, training_loss=0.2167706063217894, metrics={'train_runtime': 18108.6224, 'train_samples_per_second': 46.327, 'train_steps_per_second': 0.724, 'total_flos': 2.2072702167416832e+17, 'train_loss': 0.2167706063217894, 'epoch': 4.0})

In [27]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.19651786983013153, 'eval_roc_auc': 0.9390734016146388, 'eval_accuracy': 0.9390830027464144, 'eval_runtime': 337.5056, 'eval_samples_per_second': 155.352, 'eval_steps_per_second': 2.43, 'epoch': 4.0}


In [ ]:
# Also evaluate them (and logistic regression） on new data